In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statistics
from wordcloud import WordCloud, STOPWORDS

In [ ]:
data50=pd.read_csv('../input/restaurant-business-rankings-2020/Future50.csv')
data50.head()

In [ ]:
data250=pd.read_csv('../input/restaurant-business-rankings-2020/Top250.csv')
data250

In [ ]:
data100=pd.read_csv('../input/restaurant-business-rankings-2020/Independence100.csv')
data100.head()

In [ ]:
data50.replace({'%':''}, regex=True, inplace=True) 
data50['YOY_Sales'] = data50['YOY_Sales'].astype(float)
data50['YOY_Units'] = data50['YOY_Units'].astype(float)
data50.head()

In [ ]:
data250.replace({'%':''}, regex=True, inplace=True) 
data250['YOY_Sales'] = data50['YOY_Sales'].astype(float)
data250['YOY_Units'] = data50['YOY_Units'].astype(float)
data250.head()

# **what is YOY_Units?**

* **Future50**

In [ ]:
data50.dtypes

In [ ]:
number_type=data50.loc[:, data50.dtypes != np.object]
number_type.head()

In [ ]:
object_type=data50.loc[:, data50.dtypes == np.object]
object_type.head()

In [ ]:
plt.figure(figsize=(12,12))
sns.pairplot(data50, x_vars=number_type.columns, y_vars=["YOY_Units"], height=5, aspect=.8, kind="reg")

In [ ]:
sns.displot(data50, x="YOY_Units", hue="Franchising", kind="ecdf")

In [ ]:
data50.Location.unique()

Separate Location to City and State

In [ ]:
new = data50["Location"].str.split(",", n = 1, expand = True) 
data50["City"]= new[0] 
data50["State"]= new[1] 
data50.drop(columns =["Location"], inplace = True) 
data50.head()

In [ ]:
sns.jointplot(data=data50, x="YOY_Units", y="City")

In [ ]:
sns.jointplot(data=data50, x="YOY_Units", y="State")

* **Top250**

In [ ]:
data250.dtypes

In [ ]:
number250_type=data250.loc[:, data250.dtypes != np.object]
print("Number type columns : ",number250_type.columns)
print(" ")
object250_type=data250.loc[:, data250.dtypes == np.object]
print("Object type columns : ",object250_type.columns)

In [ ]:
for i in data250.columns:
    if data250[i].isnull().sum()>0:
        print("Have null",i,"column : ",str((data250[i].isnull().sum()/len(data250))*100)+"%")

a lot of null value T3T

In [ ]:
plt.figure(figsize=(12,12))
sns.pairplot(data250, x_vars=number250_type.columns, y_vars=["YOY_Units"], height=5, aspect=.8, kind="reg")

# **Small businesses versus large companies?**
compare Units with other Attribute

* Future50

I will seperate Small businesses and Large companies with mean "Units" value

In [ ]:
sns.displot(data50, x="Units", kind="kde")

In [ ]:
mean_units = statistics.mean(data50['Units']) 
mean_units

In [ ]:
def change_data(row):
    if row['Units']>=34.7: return "Large"
    elif row['Units']<34.7: return "Small"
    else: return 1

In [ ]:
data50.loc[:, 'Units']=data50.apply(change_data, axis=1)

In [ ]:
data50.dtypes

In [ ]:
sns.displot(data50, x="YOY_Sales", hue="Units", multiple="dodge")
plt.title('Compare Units and YOY_Sales')

In [ ]:
sns.displot(data50, x="Sales", hue="Units", multiple="dodge")
plt.title('Compare Units and Sales')

In [ ]:
plt.figure(figsize=(12,12))
sns.countplot(y="City", hue="Units", data=data50)
plt.title('Compare Units and City')

In [ ]:
plt.figure(figsize=(12,12))
sns.countplot(y="State", hue="Units", data=data50)
plt.title('Compare Units and State')

In [ ]:
sns.displot(data50, x="Franchising", hue="Units", multiple="dodge")
plt.title('Compare Units and Franchising')

In [ ]:
plt.figure(figsize=(5,5))
sns.stripplot(y='YOY_Sales', x='Franchising',data=data50, jitter=True, marker='o',alpha=0.8,hue='Units')
sns.jointplot(data=data50, x="Units", y="YOY_Sales")

In [ ]:
plt.figure(figsize=(5,5))
sns.stripplot(y='Sales', x='Franchising',data=data50, jitter=True, marker='o',alpha=0.8,hue='Units')
sns.jointplot(data=data50, x="Units", y="Sales")

# **Find other interesting information**

*** Word Clound**

In [ ]:
data250.Segment_Category.unique()

In [ ]:
data250.Segment_Category.replace({'&':'','/':""}, regex=True, inplace=True) 

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
for val in data250.Segment_Category: 
      
    val = str(val) 
  
    tokens = val.split() 
      
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
                    
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

I will split Segment_Category to restaurant type(ex. quick service, Fast casual, Bar) and food type(ex. Burger, Chicken)

In [ ]:
all_cat=[]
for i in data250.Segment_Category.unique():
    x=i.split(" ")
    for text in x:
        if text not in all_cat:
            all_cat.append(text)

see all words

In [ ]:
all_cat

In [ ]:
data250["Res_type"]=data250.Segment_Category
data250["Food_type"]=data250.Segment_Category

In [ ]:
data250.head()

In [ ]:
res_list = ['Cafe', 'Service','Dining','Bar','Casual','Quick','FamilyStyle','Family']
list_difference = [item for item in all_cat if item not in res_list]
print(list_difference)

Split Res_type

In [ ]:
words=data250["Res_type"]
res_type_data=[]
for i in words:
    sc = set(list_difference)
    i=i.split(" ")
    i=''.join([c for c in i if c not in sc])
    res_type_data.append(i)
data250["Res_type"]=res_type_data
data250["Res_type"].unique()

Split Food_type

In [ ]:
res_list.append('Sports')
res_list.append('Style')

In [ ]:
words=data250["Food_type"]
food_type_data=[]
for i in words:
    sc = set(res_list)
    i=i.split(" ")
    i=''.join([c for c in i if c not in sc])
    food_type_data.append(i)
data250["Food_type"]=food_type_data
data250["Food_type"].unique()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(y="Food_type", data=data250)

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(y="Res_type", data=data250)